In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime
from numpy import array


     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 8.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 1.7 MB/s 


In [3]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
'all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(15,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-15][0],
                df[each-15][1],
                df[each-15][2],
                df[each-15][3],
                df[each-15][4],
                df[each-14][0],
                df[each-14][1],
                df[each-14][2],
                df[each-14][3],
                df[each-14][4],
                df[each-13][0],
                df[each-13][1],
                df[each-13][2],
                df[each-13][3],
                df[each-13][4],
                df[each-12][0],
                df[each-12][1],
                df[each-12][2],
                df[each-12][3],
                df[each-12][4],
                df[each-11][0],
                df[each-11][1],
                df[each-11][2],
                df[each-11][3],
                df[each-11][4],
                df[each-10][0],
                df[each-10][1],
                df[each-10][2],
                df[each-10][3],
                df[each-10][4],
                df[each-9][0],
                df[each-9][1],
                df[each-9][2],
                df[each-9][3],
                df[each-9][4],
                df[each-8][0],
                df[each-8][1],
                df[each-8][2],
                df[each-8][3],
                df[each-8][4],
                df[each-7][0],
                df[each-7][1],
                df[each-7][2],
                df[each-7][3],
                df[each-7][4],
                df[each-6][0],
                df[each-6][1],
                df[each-6][2],
                df[each-6][3],
                df[each-6][4],
                df[each-5][0],
                df[each-5][1],
                df[each-5][2],
                df[each-5][3],
                df[each-5][4],
                df[each-4][0],
                df[each-4][1],
                df[each-4][2],
                df[each-4][3],
                df[each-4][4],
                df[each-3][0],
                df[each-3][1],
                df[each-3][2],
                df[each-3][3],
                df[each-3][4],
                df[each-2][0],
                df[each-2][1],
                df[each-2][2],
                df[each-2][3],
                df[each-2][4],
                df[each-1][0],
                df[each-1][2],
                df[each-1][1],
                df[each-1][3],
                df[each-1][4],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(1,10))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["suggestion"] = last_column
  return scaled
def column_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  df_scaled = scaler.fit_transform(df.to_numpy())
  df_scaled = pd.DataFrame(df_scaled)
  df_scaled["suggestion"] = last_column
  return df_scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      #df = column_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df   

In [4]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

symbols = get_crypto_syms()
#symbols = read_syms_from_txt()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="730d",interval="1h",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 16:
          dfs.append(df)

data = process(dfs)
xTrain, xTest, yTrain, yTest = spliting(data)
data

(1470591, 75) (1470591,)
(163399, 75) (163399,)


,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,suggestion
0,1.000000,2.413078,1.000000,2.397932,2.397932,2.392553,3.030922,2.392553,2.467638,2.467638,...,9.864683,4.463093,9.575571,9.575571,9.573407,8.672007,10.000000,9.408121,9.408121,0.0
1,1.111501,1.842335,1.111501,1.197462,1.197462,1.000000,3.178040,1.000000,2.449449,2.449449,...,9.820848,8.300502,9.143238,9.143238,9.461206,8.712966,10.000000,9.417154,9.417154,0.0
2,1.000000,3.139220,1.000000,2.423615,2.423615,2.408295,3.254652,2.135184,2.706063,2.706063,...,9.839592,8.575497,9.267134,9.267134,9.590745,8.165913,10.000000,9.227517,9.227517,1.0
3,1.973256,2.862124,1.686427,2.285981,2.285981,2.072318,2.670905,1.000000,2.233704,2.233704,...,9.946274,8.020065,9.134991,9.134991,9.038035,7.788862,10.000000,9.165100,9.165100,1.0
4,2.066170,2.661325,1.000000,2.226631,2.226631,2.125288,3.830624,2.086975,3.651178,3.651178,...,9.948401,7.749940,9.118288,9.118288,9.079426,8.343790,10.000000,9.287950,9.287950,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1633985,3.401987,4.702387,1.969713,2.035480,2.035480,1.932244,3.121504,1.931024,2.809162,2.809162,...,5.313361,3.491221,3.671835,3.671835,3.663833,2.874392,3.800440,2.986556,2.986556,1.0
1633986,1.932244,3.121504,1.931024,2.809162,2.809162,2.849754,4.094730,2.517214,3.526787,3.526787,...,3.800440,2.874392,2.986556,2.986556,2.994997,2.490771,9.051900,4.630229,4.630229,0.0
1633987,2.849754,4.094730,2.517214,3.526787,3.526787,4.014571,6.739829,3.681641,6.739829,6.739829,...,9.051900,2.490771,4.630229,4.630229,4.650720,2.723442,5.598235,3.062910,3.062910,0.0
1633988,4.014571,6.739829,3.681641,6.739829,6.739829,6.841064,8.210450,6.395775,7.879276,7.879276,...,5.598235,2.723442,3.062910,3.062910,2.699389,1.712990,4.202942,1.712990,1.712990,0.0


In [6]:
model = Sequential()


model.add(Dense(1500, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 1500)              114000    
                                                                 
 dense_6 (Dense)             (None, 1000)              1501000   
                                                                 
 dense_7 (Dense)             (None, 500)               500500    
                                                                 
 dense_8 (Dense)             (None, 100)               50100     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 2,165,701
Trainable params: 2,165,701
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(xTrain,yTrain,epochs=50,batch_size=50000,validation_data=(xTest,yTest))

Epoch 1/50
 3/30 [==>...........................] - ETA: 4:48 - loss: 0.7110 - accuracy: 0.5063

KeyboardInterrupt: ignored

In [ ]:
model.save("9474_BinaryTest_v08.h5")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

data = pd.read_csv("/content/drive/MyDrive/Colab Files/BinaryTest_v10_hourtesting.csv")
data = data.drop("Unnamed: 0",axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

data.to_csv("BinaryTest_v10_hourtesting.csv")